In [20]:
import numpy as np
import pandas as pd

### PARTIE 1 : Exploration de la base de données

In [21]:
df = pd.read_csv('csv_files/regularite-mensuelle-tgv-aqst.csv',sep=';')
missing_data = df.isnull().sum()
print('Donées manquantes :', '\n', missing_data[missing_data>0], '\n')
print('Type des données :', '\n', df.dtypes)

Donées manquantes : 
 commentaire_annulation         8154
commentaire_retards_depart     8154
commentaires_retard_arrivee    7456
dtype: int64 

Type des données : 
 date                                     object
service                                  object
gare_depart                              object
gare_arrivee                             object
duree_moyenne                             int64
nb_train_prevu                            int64
nb_annulation                             int64
commentaire_annulation                  float64
nb_train_depart_retard                    int64
retard_moyen_depart                     float64
retard_moyen_tous_trains_depart         float64
commentaire_retards_depart              float64
nb_train_retard_arrivee                   int64
retard_moyen_arrivee                    float64
retard_moyen_tous_trains_arrivee        float64
commentaires_retard_arrivee              object
nb_train_retard_sup_15                    int64
retard_moyen_train

In [22]:
print('Information : ', '\n', df.info())
print('Description statistique de chaque colonne : ', '\n', df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  8154 non-null   object 
 1   service                               8154 non-null   object 
 2   gare_depart                           8154 non-null   object 
 3   gare_arrivee                          8154 non-null   object 
 4   duree_moyenne                         8154 non-null   int64  
 5   nb_train_prevu                        8154 non-null   int64  
 6   nb_annulation                         8154 non-null   int64  
 7   commentaire_annulation                0 non-null      float64
 8   nb_train_depart_retard                8154 non-null   int64  
 9   retard_moyen_depart                   8154 non-null   float64
 10  retard_moyen_tous_trains_depart       8154 non-null   float64
 11  commentaire_retar

### PARTIE 2 : Traitement de la base de données
#### 1 : Gestion des commentaires

On observe que les seules valeurs manquantes sont pour les features commentaires : commentaire_annulation, commentaire_retards_depart
et commentaire_retard_arrivee.
Pour une première approche simplificatrice du dataset, on prend la décision de supprimier les features commentaires.

In [23]:
df_clean = (
    df.copy()
        .drop(
            ["commentaire_annulation", "commentaire_retards_depart", "commentaires_retard_arrivee"],
            axis=1,
        )
)

df_clean.columns

Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'nb_train_depart_retard',
       'retard_moyen_depart', 'retard_moyen_tous_trains_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'nb_train_retard_sup_15',
       'retard_moyen_trains_retard_sup15', 'nb_train_retard_sup_30',
       'nb_train_retard_sup_60', 'prct_cause_externe', 'prct_cause_infra',
       'prct_cause_gestion_trafic', 'prct_cause_materiel_roulant',
       'prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

#### 2 : Gestion de la colonne service

In [24]:
df_clean["service"].unique()

array(['National', 'International'], dtype=object)

On observe que la colonne ne possède que deux valeurs categorielles, nous allons faire un "one-hot" encoding et creer une feature booléenne "national_service"

In [25]:
df_clean["national_service"] = np.where(df_clean["service"]=="National", 1,0)
df_clean.drop("service", axis=1, inplace=True)

#### 3 : Gestion de la date
Nous allons créer deux colonnes, une pour le mois et l'autre pour l'année.


In [26]:
df_clean[['annee', 'mois']] = df_clean['date'].str.split('-', expand=True)
df_clean.drop("date", axis=1, inplace=True)
df_clean


,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,nb_train_retard_arrivee,retard_moyen_arrivee,...,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,national_service,annee,mois
0,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,289,11.247809,3.693179,147,28.436735,...,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336,1,2018,01
1,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,8,2.875000,0.095796,34,21.524020,...,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000,1,2018,01
2,PARIS MONTPARNASSE,QUIMPER,220,248,1,37,9.501351,1.003981,26,55.692308,...,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000,1,2018,01
3,PARIS MONTPARNASSE,ST MALO,156,102,0,12,19.912500,1.966667,13,48.623077,...,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000,1,2018,01
4,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,61,7.796995,0.886889,71,12.405164,...,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000,1,2018,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,STRASBOURG,PARIS EST,114,492,2,273,11.989927,6.742041,105,30.157302,...,10,7.619048,5.714286,55.238095,18.095238,7.619048,5.714286,1,2023,06
8150,TOULOUSE MATABIAU,PARIS MONTPARNASSE,273,215,0,24,28.730556,3.181240,36,76.688889,...,10,13.888889,33.333333,8.333333,19.444444,8.333333,16.666667,1,2023,06
8151,TOURS,PARIS MONTPARNASSE,78,192,1,20,31.281667,3.276353,32,45.841146,...,5,14.285714,21.428571,28.571429,21.428571,3.571429,10.714286,1,2023,06
8152,VALENCE ALIXAN TGV,PARIS LYON,133,440,2,347,13.517051,10.271499,116,41.487213,...,22,26.724138,19.827586,27.586207,7.758621,7.758621,10.344828,1,2023,06


#### 4 : Gestion des villes
La dernière étape du preprocessing est celle du traitement des colonnes gare_depart et gare_arrivee. Pour garde l'information gare de départ ou d'arrivée, on crée une colonne par ville de départ, avec le préfixe DEPART, et de même avec les villes d'arrivée, avec le préfixe ARRIVEE. On décide ainsi de garder la visibilité sur les caractéristiques de départ et d'arrivée des gares, car en effet la gare du départ ou d'arrivée peut avoir un impact sur le retard indépendemment du trajet.

In [27]:
depart = pd.get_dummies(df_clean['gare_depart'],prefix='DEPART') # Créer une colonne par ville de départ
df_clean= pd.concat([df_clean,depart], axis=1).drop('gare_depart', axis=1) ## Concaténer les colonnes et supprimer la colonne avec les données catégorilles
arrivee = pd.get_dummies(df_clean['gare_arrivee'],prefix='ARRIVEE') # Créer une colonne par ville d'arrivée'
df_clean= pd.concat([df_clean,arrivee], axis=1).drop('gare_arrivee', axis=1) 
df_clean 

,duree_moyenne,nb_train_prevu,nb_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,nb_train_retard_arrivee,retard_moyen_arrivee,retard_moyen_tous_trains_arrivee,nb_train_retard_sup_15,...,ARRIVEE_ST PIERRE DES CORPS,ARRIVEE_STRASBOURG,ARRIVEE_STUTTGART,ARRIVEE_TOULON,ARRIVEE_TOULOUSE MATABIAU,ARRIVEE_TOURCOING,ARRIVEE_TOURS,ARRIVEE_VALENCE ALIXAN TGV,ARRIVEE_VANNES,ARRIVEE_ZURICH
0,141,870,5,289,11.247809,3.693179,147,28.436735,6.511118,110,...,0,0,0,0,0,0,0,0,0,0
1,165,222,0,8,2.875000,0.095796,34,21.524020,5.696096,22,...,0,0,0,0,0,0,0,0,0,0
2,220,248,1,37,9.501351,1.003981,26,55.692308,7.578947,26,...,0,0,0,0,0,0,0,0,0,0
3,156,102,0,12,19.912500,1.966667,13,48.623077,6.790686,8,...,0,0,0,0,0,0,0,0,0,0
4,61,391,2,61,7.796995,0.886889,71,12.405164,3.346487,17,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,114,492,2,273,11.989927,6.742041,105,30.157302,7.689898,80,...,0,0,0,0,0,0,0,0,0,0
8150,273,215,0,24,28.730556,3.181240,36,76.688889,14.824264,36,...,0,0,0,0,0,0,0,0,0,0
8151,78,192,1,20,31.281667,3.276353,32,45.841146,8.650349,13,...,0,0,0,0,0,0,0,0,0,0
8152,133,440,2,347,13.517051,10.271499,116,41.487213,12.765753,96,...,0,0,0,0,0,0,0,0,0,0


### PARTIE 3 : Ajout de nouveaux features intéressants

Dans notre objectif de prédire les retards des trains TGV à l'arrivée, certaines features nous semblent intéressants à ajouter au dataset : 
- Le **nombre de voies par gare**. On pourra ainsi chercher une relation entre les retards d'une gare et le nombre de voies qu'elle possède.
- Les mois possédant des semaines de **vacances scolaires**. On cherche ici une relation possible entre les mois à plus forts retards et l'affluence en gare / fréquentation des transports ferroviaires sur lces periodes de vacances.
- Les mois durant lesquels des **grèves** ont eu lieues et son annoncées. Les grèves font partie des causes de retard à prendre en compte dans notre étude.
- La **distance entre deux gares**. Y a-t'il plus de retards sur les longs trajets ? Existe-t'il une relation entre distance entre deux gares et retards sur cette ligne ?
- Les **coordonnées GPS des gares**. En remplaçant les noms des gares par leurs coordonnées GPS, on pourrait plus facilement accéder à la distance entre deux gares. 

 #### 1 : Ajouter le nombre de voies par gare

On commence par créer un fichier *nbdevoies.csv* contenant le nombre de voies pour chaque gare du dataset. 
Tout d'abord, on récupère la liste des gares présentes dans le dataset. Puis, pour chacune d'entre elles, on recherche le nombre de voies qu'elle possède. Les données sont obtenues par recherches personnelles.
Dans les gares, les passagers sont sur les quais et les trains sur les voies. En comptant le nombre de voies, on peut avoir une idée du nombre maximal de trains qu'il est possible d'avoir en gare en même temps.

In [28]:
# Obtenir la liste des villes présentes comme gare de départ et gare d'arrivée
df_clean["gare_depart"].unique()
df_clean["gare_arrivee"].unique()

KeyError: 'gare_depart'

In [ ]:
nb_voies = pd.read_csv('csv_files/nbdevoies.csv',sep=',')

# Vérifier que le dataset est bien complet 
print('Donées manquantes :', '\n', nb_voies.isnull().sum(), '\n')
print('Type des données :', '\n', nb_voies.dtypes)
nb_voies

Donées manquantes : 
 Gare               0
Nombre de voies    0
dtype: int64 

Type des données : 
 Gare               object
Nombre de voies     int64
dtype: object


,Gare,Nombre de voies
0,Paris-Lyon,14
1,Paris-Montparnasse,12
2,Paris-Nord,22
3,Tourcoing,4
4,Paris-Vaugirard,10
5,Paris-Est,12
6,Lyon-Part-Dieu,18
7,Marseille-Saint-Charles,10
8,Rennes,6
9,Lille,10


Après l'importation de ce fichier *nb_voies*, on intègre les valeurs au dataset *data_clean* pour chaque gare correspondante. On appelera *nb_voies_depart* et *nb_voies_arrivee* resepctivement les nombres de voies dans les gares de départ et d'arrivée d'un trajet. 

In [ ]:
# Fusionnez les deux dataframe df_clean et nb_quais selon les noms des gares de départ et d'arrivée

df_clean = df_clean.merge(nb_voies, left_on='gare_depart', right_on='Gare', how='left')
df_clean.rename(columns={'nb_voies': 'nb_voies_depart'}, inplace=True)
df_clean = df_clean.merge(nb_voies, left_on='gare_arrivee', right_on='Gare', how='left')
df_clean.rename(columns={'nb_voies': 'nb_voies_arrivee'}, inplace=True)
df_clean.drop(['Gare_x', 'Gare_y'], axis=1, inplace=True)
df_clean


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Gare_x', 'Nombre de voies_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,nb_train_retard_arrivee,retard_moyen_arrivee,...,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,national_service,annee,mois,Nombre de voies_x,Nombre de voies_y,Nombre de voies_x,Nombre de voies_y
0,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,289,11.247809,3.693179,147,28.436735,...,15.966387,5.042017,0.840336,1,2018,01,NaN,NaN,NaN,NaN
1,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,8,2.875000,0.095796,34,21.524020,...,11.538462,3.846154,0.000000,1,2018,01,NaN,NaN,NaN,NaN
2,PARIS MONTPARNASSE,QUIMPER,220,248,1,37,9.501351,1.003981,26,55.692308,...,19.230769,0.000000,0.000000,1,2018,01,NaN,NaN,NaN,NaN
3,PARIS MONTPARNASSE,ST MALO,156,102,0,12,19.912500,1.966667,13,48.623077,...,15.384615,7.692308,0.000000,1,2018,01,NaN,NaN,NaN,NaN
4,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,61,7.796995,0.886889,71,12.405164,...,21.212121,6.060606,0.000000,1,2018,01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,STRASBOURG,PARIS EST,114,492,2,273,11.989927,6.742041,105,30.157302,...,18.095238,7.619048,5.714286,1,2023,06,NaN,NaN,NaN,NaN
8150,TOULOUSE MATABIAU,PARIS MONTPARNASSE,273,215,0,24,28.730556,3.181240,36,76.688889,...,19.444444,8.333333,16.666667,1,2023,06,NaN,NaN,NaN,NaN
8151,TOURS,PARIS MONTPARNASSE,78,192,1,20,31.281667,3.276353,32,45.841146,...,21.428571,3.571429,10.714286,1,2023,06,NaN,NaN,NaN,NaN
8152,VALENCE ALIXAN TGV,PARIS LYON,133,440,2,347,13.517051,10.271499,116,41.487213,...,7.758621,7.758621,10.344828,1,2023,06,NaN,NaN,NaN,NaN


#### 2 : Ajouter les vacances scolaires

On souhaite désormais marquer chaque mois contenant des semaines de vacances scolaires depuis le mois de janvier 2018 jusqu'au mois de juin 2023. On repère par 1 un mois contenant des vacances, et par 0 un mois sans.

In [ ]:
vac = pd.read_csv('csv_files/vacances_scolaires.csv',sep=';')

# Vérifier que le dataset est bien complet 
print('Donées manquantes :', '\n', vac.isnull().sum(), '\n')
print('Type des données :', '\n', vac.dtypes)
vac

FileNotFoundError: [Errno 2] No such file or directory: 'csv_files/vacances_scolaires.csv'

In [ ]:
df_clean['annee'] = df_clean['annee'].astype(int)
df_clean['mois'] = df_clean['mois'].astype(int)
df_clean = df_clean.merge(vac, on=['annee', 'mois'], how='left')
df_clean

### PARTIE 4 : Exportation de la base de donnée traitée

A faire : 
Sortir une base de donnée par application
- une sans les prct
- une sans trop de colonnes pour prédire le retard moyen

A reprendre : 
- format des dates

In [29]:
df_clean.to_csv('/Users/marche/Documents/Centrale 3A/Apprentissage_Automatique_Project/Python_files/csv_files/retard_train_bdd_clean.csv', index=False)